In [3]:
import logging
import os
import sys

sys.path.append("../")

import PIL.Image as Image

import ignite
import ignite.distributed as idist
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
from src.DCGAN import Discriminator, Generator, get_noise
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Engine, Events
from ignite.metrics import FID, InceptionScore, RunningAverage, SSIM

from torchsummary import summary
from torchvision.datasets import ImageFolder

ModuleNotFoundError: No module named 'src.DCGAN'

In [7]:
latent_dim = 100
n_channels = 3
image_size = 64

In [37]:
class Discriminator(nn.Module):
    """
    DCGAN Discriminator
    """

    def __init__(
        self,
        features_dim=128,
        img_channels=3,
        kernel_size=4,
        stride=2,
        padding=1,
    ):
        """
        Discriminator Class

        Parameters
        ----------
        img_channels: int
            number if channels in image
        features_dim: int
            inner hidden dimension

        Returns
        ----------
        none
        """

        super(Discriminator, self).__init__()

        self.net = nn.Sequential(
            # img_channels x 64 x 64
            nn.Conv2d(img_channels, features_dim, kernel_size, stride, padding),
            nn.LeakyReLU(0.2, inplace=True),
            # features_dim x 32 x 32
            nn.Conv2d(
                features_dim, features_dim * 2, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # (features_dim * 2) x 16 x 16
            nn.Conv2d(
                features_dim * 2, features_dim * 4, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # (features_dim * 4) x 8 x 8
            nn.Conv2d(
                features_dim * 4, features_dim * 8, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # (features_dim * 8) x 4 x 4
            nn.Conv2d(
                features_dim * 8, features_dim * 16, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # (features_dim * 16) x 2 x 2
            nn.Conv2d(features_dim * 16, 1, kernel_size, stride=1, padding=0),
            # 1 x 1 x 1
            nn.Sigmoid(),
        )

    def forward(self, image):
        """
        Function for completing a forward pass of the discriminator: Given an image tensor,
        returns a 1-dimension tensor representing fake/real.

        Parameters
        ----------
        image: tensor
            a flattened image tensor with dimension (im_dim)

        Returns
        ----------
        _: tensor
            real or fake
        """
        return self.net(image)

image_size = 128
netD = idist.auto_model(Discriminator())
summary(netD, (n_channels, image_size, image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 64, 64]           6,272
         LeakyReLU-2          [-1, 128, 64, 64]               0
            Conv2d-3          [-1, 256, 32, 32]         524,544
       BatchNorm2d-4          [-1, 256, 32, 32]             512
         LeakyReLU-5          [-1, 256, 32, 32]               0
            Conv2d-6          [-1, 512, 16, 16]       2,097,664
       BatchNorm2d-7          [-1, 512, 16, 16]           1,024
         LeakyReLU-8          [-1, 512, 16, 16]               0
            Conv2d-9           [-1, 1024, 8, 8]       8,389,632
      BatchNorm2d-10           [-1, 1024, 8, 8]           2,048
        LeakyReLU-11           [-1, 1024, 8, 8]               0
           Conv2d-12           [-1, 2048, 4, 4]      33,556,480
      BatchNorm2d-13           [-1, 2048, 4, 4]           4,096
        LeakyReLU-14           [-1, 204

In [34]:
class Generator(nn.Module):
    """
    DCGAN Generator
    """

    def __init__(
        self,
        z_dim,
        features_dim=128,
        img_channels=3,
        kernel_size=4,
        stride=2,
        padding=1,
    ):
        """
        Generator Class

        Parameters
        ----------
        z_dim: int
            the dimension of the noise vector
        img_channels: int
            number if channels in image
        features_dim: int
            inner hidden dimension

        Returns
        ----------
        none
        """

        super(Generator, self).__init__()

        self.net = nn.Sequential(
            # z_dim x 1 x 1
            nn.ConvTranspose2d(
                z_dim, features_dim * 16, kernel_size, stride=1, padding=0
            ),
            nn.BatchNorm2d(features_dim * 16),
            nn.ReLU(inplace=True),
            # (features_dim * 16) x 4 x 4
            nn.ConvTranspose2d(
                features_dim * 16,
                features_dim * 8,
                kernel_size,
                stride,
                padding,
            ),
            nn.BatchNorm2d(features_dim * 8),
            nn.ReLU(inplace=True),
            # (features_dim * 8) x 8 x 8
            nn.ConvTranspose2d(
                features_dim * 8, features_dim * 4, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 4),
            nn.ReLU(inplace=True),
            # (features_dim * 4) x 16 x 16
            nn.ConvTranspose2d(
                features_dim * 4, features_dim * 2, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim * 2),
            nn.ReLU(inplace=True),
            # (features_dim * 2) x 32 x 32
            nn.ConvTranspose2d(
                features_dim * 2, features_dim, kernel_size, stride, padding
            ),
            nn.BatchNorm2d(features_dim),
            nn.ReLU(inplace=True),
            # (features_dim) x 64 x 64
            nn.ConvTranspose2d(
                features_dim, img_channels, kernel_size, stride, padding
            ),
            # img_channels x 128 x 128
            nn.Tanh(),
        )

    def forward(self, image):
        """
        Function for completing a forward pass of the generator: Given a noise tensor,
        returns generated images.

        Parameters
        ----------
        noise: tensor
            a noise tensor with dimensions (n_samples, z_dim, 1, 1)

        Returns
        ----------
        _: image
            image after a forward pass
        """
        return self.net(image)

latent_dim = 256
netG = idist.auto_model(Generator(latent_dim))
summary(netG, (latent_dim, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 2048, 4, 4]       8,390,656
       BatchNorm2d-2           [-1, 2048, 4, 4]           4,096
              ReLU-3           [-1, 2048, 4, 4]               0
   ConvTranspose2d-4           [-1, 1024, 8, 8]      33,555,456
       BatchNorm2d-5           [-1, 1024, 8, 8]           2,048
              ReLU-6           [-1, 1024, 8, 8]               0
   ConvTranspose2d-7          [-1, 512, 16, 16]       8,389,120
       BatchNorm2d-8          [-1, 512, 16, 16]           1,024
              ReLU-9          [-1, 512, 16, 16]               0
  ConvTranspose2d-10          [-1, 256, 32, 32]       2,097,408
      BatchNorm2d-11          [-1, 256, 32, 32]             512
             ReLU-12          [-1, 256, 32, 32]               0
  ConvTranspose2d-13          [-1, 128, 64, 64]         524,416
      BatchNorm2d-14          [-1, 128,

In [1]:
"""
Discriminator and Generator implementation from DCGAN paper
"""

import torch
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)



N, in_channels, H, W = 8, 3, 64, 64
noise_dim = 100
x = torch.randn((N, in_channels, H, W))
disc = Discriminator(in_channels, 8)
assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
gen = Generator(noise_dim, in_channels, 8)
z = torch.randn((N, noise_dim, 1, 1))
assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"

In [21]:
disc(x).shape

torch.Size([8, 1, 1, 1])

In [20]:
disc(x).reshape(-1).shape

torch.Size([8])

In [26]:
gen(z).shape

torch.Size([8, 3, 64, 64])

In [27]:
gen(z).reshape(-1).shape

torch.Size([98304])

In [22]:
N, in_channels, H, W = 64, 3, 64, 64
noise_dim = 100
x = torch.randn((N, in_channels, H, W))
disc = Critic(in_channels)
assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
gen = Generator(noise_dim, in_channels)
z = torch.randn((N, noise_dim, 1, 1))
assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"

AssertionError: Discriminator test failed

In [4]:
from src.WGANx64 import Critic, Generator

In [5]:
disc = Critic()
gen = Generator(noise_dim)

In [6]:
disc(x).shape

torch.Size([8, 1, 1, 1])

In [7]:
disc(x).reshape(-1).shape

torch.Size([8])

In [8]:
gen(z).shape

torch.Size([8, 3, 64, 64])

In [9]:
gen(z).reshape(-1).shape

torch.Size([98304])